In [ ]:
import os

from glob import glob

from bs4 import BeautifulSoup

import IPython

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson import SpeechToTextV1

from ibm_watson import AssistantV2

from ibm_watson import TextToSpeechV1

recognition_service = SpeechToTextV1(IAMAuthenticator('{YOUR_APIKEY}'))
recognition_service.set_service_url('{YOUR_ENDPOINT}')
SPEECH_EXTENSION = "*.webm"
SPEECH_AUDIOTYPE = "audio/webm"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]


assistant = AssistantV2(version='2019-02-28', authenticator=IAMAuthenticator('{YOUR_APIKEY}'))
assistant.set_service_url('{YOUR_ENDPOINT}')
ASSISTANT_ID = "{YOUR_ASSISTANT_ID}"
session_id = assistant.create_session(assistant_id =ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

synthesis_service = TextToSpeechV1(IAMAuthenticator('{YOUR_APIKEY}'))
synthesis_service.set_service_url('{YOUR_ENDPOINT}')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.wav", autoplay=True)



speak_text(message_assistant(recognize_audio()))
